In [1]:
import pandas as pd 
import numpy as np
import os 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf 
from tensorflow import keras 
# from tensorflow.keras import layers 
import tensorflow_hub as hub 
# from own_dataset import get_loaders
from tqdm import tqdm 

In [2]:
import os 
from tqdm import tqdm 
import torchvision.datasets as datasets
from torch.utils.data import WeightedRandomSampler ,DataLoader
import torchvision.transforms as transforms

def get_loaders(train_dir , batch_size , image_size ):
    print ('getting loaders')
    train_transforms = transforms.Compose(
        [
            transforms.Resize((300,300)),
            transforms.RandomCrop((image_size,image_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.05),
            transforms.ToTensor(),
        ]
    )
    # dev_transforms = transforms.Compose([
    #     transforms.Resize((image_size , image_size)) ,
    #     transforms.ToTensor() ,
        
    # ])
    train_dataset = datasets.ImageFolder(root = train_dir ,
                                         transform = train_transforms
                                        )
    # dev_dataset = datasets.ImageFolder(root = dev_dir ,
    #                                   transform = dev_transfrom)
    # val_loader = DataLoader(dev_dataset , batch_size =batch_size , num_workers= 2 , pin_memory=True )
    class_weights =[]
    for root , subdir , files in os.walk(train_dir):
        if len (files) > 0:
            class_weights.append(1/len(files))

    sample_weights =[0] *len(train_dataset)
    for idx , (data , labels) in enumerate(tqdm(train_dataset.imgs)):
        class_weight =class_weights[labels]
        sample_weights[idx] = class_weight

    sampler = WeightedRandomSampler(sample_weights , num_samples = len(sample_weights) , replacement = True)
    train_loader = DataLoader(train_dataset , batch_size = batch_size , sampler =sampler ,num_workers= 2 , pin_memory=True )
    return train_loader 
   

C:\Users\amrsa\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
if len (physical_devices) > 0 :
    tf.config.expremintal.set_memory_growth(physsical_devices[0] ,True)


In [4]:
URL = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 120
NUM_EPOCHS = 3
DATA_DIR = "data/images/images"
MODEL_PATH = "efficientb0/"
LOAD_MODEL = False

os.environ["TFHUB_DOWNLOAD_PROGRESS"] ="1"
os.environ["TFHUB_CACHE_DIR"] ="C://users/amrsa/save_b0_models"


In [5]:
def get_model(url, img_size, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(img_size, img_size, 3)), 
        tf.keras.layers.Lambda(lambda x: hub.KerasLayer(url)(x), trainable=True),  
        tf.keras.layers.Dense(1000, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax'),
    ])
    return model

In [6]:
@tf.function
def train_step(data , labels , acc_metrics , model , loss_fn ,optimizer ):
    # data = data.premute(0 , 2 , 3 ,1)
    # data = tf.convert_to_tensor(np.array(data))
    # labels = tf.convert_to_tensor(np.array(labels))
    with tf.GradientTape() as tape :
        predictions = model (data , training =True )
        loss = loss_fn (labels , predictions)

    gradients = tape.gradient(loss , model.trainable_weights  )
    optimizer.apply_gradients(zip(gradients , model.trainable_weights))
    acc_metrics.update_state(labels , predictions)
    

In [7]:
def evaluate_model(ds_validation):
    acc_metric = keras.metrics.SparseCategoricalAccuracy()
    for idx , (data ,labels) in enumerate(ds_validation) :
        data = tf.convert_to_tensor(np.array(data))
        labels = tf.convert_to_tensor(np.array(labels))
        y_pred = model(data , training = False)
        acc_metric.upate_state(labels , y_pred)

    acc = acc_metric.results()
    print(f'accuracy over validaion set {acc}')

In [8]:
def main():
    train_loader  = get_loaders(DATA_DIR ,  BATCH_SIZE , IMG_SIZE )
    if LOAD_MODEL :
        print ('loading model')
        model = keras.models.load_model(MODEL_PATH)

    else:
        print ('building model')
        model = get_model(URL , IMG_SIZE , NUM_CLASSES)

    optimizer = keras.optimizers.Adam(learning_rate = 3e-4)
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    acc_metrics = keras.metrics.SparseCategoricalAccuracy()

    for epoch in range(NUM_EPOCHS):
        for idx , (data ,labels) in enumerate(tqdm(train_loader)):
            data = data.permute(0 , 2 , 3 ,1)
            data = tf.convert_to_tensor(np.array(data))
            labels = tf.convert_to_tensor(np.array(labels))
            train_step(data ,labels ,acc_metrics, model , loss_fn , optimizer)

            if idx % 150 == 0 and idx > 0 :
                train_acc = acc_metrics.result()
                print (f"accuracy over epoch (so far) {train_acc}")

                # evaluate_model(dev_loader , model)
                model.save("efficientb0_model.keras")
                
        

In [9]:
if __name__ =="__main__":
    main()

getting loaders


100%|██████████| 20580/20580 [00:00<00:00, 1699723.86it/s]

building model



  0%|          | 0/644 [00:00<?, ?it/s]C:\Users\amrsa\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\nn.py:602: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
 23%|██▎       | 151/644 [01:24<03:23,  2.42it/s]

accuracy over epoch (so far) 0.5047599077224731


...

 70%|███████   | 451/644 [04:20<01:18,  2.47it/s]

accuracy over epoch (so far) 0.8050798773765564


 93%|█████████▎| 601/644 [05:21<00:17,  2.41it/s]

accuracy over epoch (so far) 0.8096601963043213


100%|██████████| 644/644 [05:39<00:00,  1.89it/s]
